sun:tf.keras.Input
sun:tf.keras.backend
sun:tf.reshape

In [7]:
import keras
import tensorflow as tf
from tensorflow import keras

## 对tensor的理解

keras.Input(shape=(1,), name='input', dtype=tf.int64) 这行代码表示创建了一个名为 input 的输入层，这个输入层有一个特征，维度为 1 维，数据类型为整数（int64）。
尽管维度是1维，但是在 TensorFlow 中，即使是一维的特征也需要被表示为张量，因此在这里使用了 (1,) 表示维度为1
shape=(1,) 中的 1 表示这个特征的维度为1，而不是表示这个特征本身是一个向量。在这种情况下，每个样本只包含一个特征值。
所以可以想象一个dataframe
      特征1
样本1   1
样本2   2
样本3   3
...

keras.Input(shape=(1,))代表的就是这样的输入层

#### 所以在定义输入层时，是不知道样本数相关概念的对吧？
在定义神经网络模型的输入层时，通常是不需要明确指定样本数量的。输入层的形状（shape）通常用来指定每个样本的特征数量，而样本的数量通常在训练时由输入数据的批次大小（batch size）确定。
在训练神经网络时，您会将整个数据集分成批次（batches）来输入模型进行训练。每个批次中包含一定数量的样本，这个数量就是批次大小。因此，样本数量是由数据集中的样本总数和批次大小共同决定的。
在定义神经网络模型时，您只需要关注每个样本的特征数量，而不需要显式地指定样本数量。样本数量在训练时会根据批次大小动态确定。

In [8]:
# shape(x,) 中x代表特征数，这个input代表有一个特征，每个样本只包含一个特征值
input = keras.Input(shape=(1,), name='input', dtype=tf.int64)

#### keras.Input(shape=(2,))的理解
keras.Input(shape=(2,), name='input', dtype=tf.int64) 这行代码表示创建了一个名为 input 的输入层，这个输入层有两个特征，每个特征的维度为 1 维，数据类型为整数（int64）。
在这种情况下，每个样本有两个特征，每个特征都是一个单独的维度为1的向量。这种表示方式适用于许多机器学习模型，包括神经网络模型，其中每个样本可以包含多个特征。
因此，shape=(2,) 中的 2 表示这个输入层有两个特征，每个特征的维度为1。这种方式适用于表示多特征的情况，每个特征都是一个单独的维度。
所以可以想象一个dataframe
      特征1  特征2
样本1   1      2
样本2   2      7
样本3   3      8
...

#### feature_column转化特征
在使用 keras.Input 定义输入层时，shape 参数中的第一个值表示特征的数量，而每个特征本身是一个一维向量。
如果您使用 feature_column 将一个数字特征分桶成了6个桶，那么可以将这个数字特征转化为6个特征来输入模型。每个特征代表一个桶，每个特征是一个一维向量。
例如，如果您有一个数字特征被分成了6个桶，您可以将这6个桶作为6个独立的特征输入到模型中，每个特征是一个一维向量，表示该特征的值。这种方式可以帮助模型更好地理解和利用原始数据的特征

In [9]:
# a特征每个样本是一个数字
data = {'a': [15, 9, 17, 19, 21, 18, 25, 30]}
a = tf.feature_column.numeric_column('a')
a_buckets = tf.feature_column.bucketized_column(a,
   boundaries=[10, 15, 20, 25, 30])
feature_layer1 = tf.keras.layers.DenseFeatures([a_buckets])
## 这里一个特征转化为6个特征
print(feature_layer1(data))    ## shape : 8 * 6

tf.Tensor(
[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]], shape=(8, 6), dtype=float32)


#### tf.tile
假设有一个形状为 (2, 2) 的张量 x：
     x = tf.constant([[1, 2], [3, 4]])
如果我们想要沿着某些维度多次复制这个张量，可以使用 tf.tile：
   tf.tile(x, [2, 3])
这将会在第一个维度上复制 2 次，在第二个维度上复制 3 次，最终得到一个形状为 (4, 6) 的张量。
注意：
1.tf.tile 不会改变张量的形状，而是复制数据。
2.复制的次数由 multiples 参数决定，这个参数的长度必须和输入张量的维度一致。
3.复制的次数可以是任意正整数，包括 0。

In [12]:
a = tf.constant([[1,2,3],[4,5,6]], tf.int32)
b = tf.constant([1,2], tf.int32)
# a的形状(2,3) b在第一个维度复制1次，第二个维度复制2次，复制完形状(2,6)
print(tf.tile(a, b))  # [[1,2,3,1,2,3],[4,5,6,4,5,6]]
c = tf.constant([2,1], tf.int32)

# a的形状(2,3) b在第一个维度复制2次，第二个维度复制1次，复制完形状(4,6)
print(tf.tile(a, c))  # [[1,2,3], [4,5,6], [1,2,3], [4,5,6]]

tf.Tensor(
[[1 2 3 1 2 3]
 [4 5 6 4 5 6]], shape=(2, 6), dtype=int32)


#### tensor做运算

In [15]:
# 定义两个张量
tensor1 = tf.constant([[1, 2], 
                       [3, 4]])
tensor2 = tf.constant([[5, 6], 
                       [7, 8]])


# 使用 tf.multiply() 函数进行相乘
result1 = tf.multiply(tensor1, tensor2)
result2 = tf.subtract(tensor1, tensor2)

# 或者直接使用 * 运算符
result1 = tensor1 * tensor2
result2 = tensor1 - tensor2
# 打印结果
print(result1)
print(result2)

tf.Tensor(
[[ 5 12]
 [21 32]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[-4 -4]
 [-4 -4]], shape=(2, 2), dtype=int32)


In [14]:
keys = tf.keras.Input(shape=(50, 32), name='keys')
print(keys.shape)
input = tf.keras.Input(shape=(32,), name='input')  # (none, 32)
input_shape = tf.keras.backend.int_shape(input) # (none, 32)
print(input_shape)
print(len(input_shape))
print('--------------------------------------')
## input数据每一个样本: [1, 2, 3, .....]  reshape后每一个样本：1个特征，每个特征32维 [[1,2,3,4,5,6 ...]]
input_reshape = tf.reshape(input, shape=[-1, 1, 32])
print(input_reshape)   ## (none, 1, 32)

input_tile = tf.tile(input_reshape, multiples=[1, 50, 1])
print(input_tile)    ## (none, 50, 32)  这个操作在特征上扩展了50倍啊 。变成了从[[1,2,3...],[1,2,3...]....]
# 做减法，形状不变
print(input_tile - keys) ## (none, 50, 32)
print(input_tile * keys) ## (none, 50, 32)


(None, 50, 32)
(None, 32)
2
--------------------------------------
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 32), dtype=tf.float32, name=None), name='tf.reshape_2/Reshape:0', description="created by layer 'tf.reshape_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50, 32), dtype=tf.float32, name=None), name='tf.tile_1/Tile:0', description="created by layer 'tf.tile_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50, 32), dtype=tf.float32, name=None), name='tf.math.subtract_1/Sub:0', description="created by layer 'tf.math.subtract_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 50, 32), dtype=tf.float32, name=None), name='tf.math.multiply/Mul:0', description="created by layer 'tf.math.multiply'")
